# Docling testing script

Extract structured data from PDF using docling.

In [ ]:
from pathlib import Path
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.settings import settings

## File path

In [ ]:
file_path = "/mnt/c/Users/User/Downloads/codes/sample pdf/pwc-my-centrestage-budget-2025.pdf"

## Extract data from PDF using docling

In [ ]:
pipeline_options = PdfPipelineOptions()
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# Enable Profiling
settings.debug.profile_pipeline_timings = True

In [ ]:
result = converter.convert(file_path)

In [ ]:
doc_conversion_secs = result.timings["pipeline_total"].times
print(f"Conversion secs: {doc_conversion_secs}")

## Create folders

In [ ]:
output_dir = Path("extracted_images/docling")
output_dir.mkdir(parents=True, exist_ok=True)

## Save pages as images

Manually save each page as image.

In [ ]:
for page_no, page in result.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"pwc-my-centrestage-budget-2025-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

## Save figures as images

Manually save the figures in the pages as images.

In [ ]:
table_counter = 0
picture_counter = 0
for element, _ in result.document.iterate_items():
    if isinstance(element, TableItem):
        table_counter += 1
        element_image_filename = (
            output_dir / f"pwc-my-centrestage-budget-2025-table-{table_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(result.document).save(fp, "PNG")

    if isinstance(element, PictureItem):
        picture_counter += 1
        element_image_filename = (
            output_dir / f"pwc-my-centrestage-budget-2025-picture-{picture_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(result.document).save(fp, "PNG")

## Save to markdown file

This function includes saving figures as images and reference them.

In [ ]:
md_filename = output_dir / "pwc-my-centrestage-budget-2025-picture-with-image-refs.md"
result.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

## Print markdown directly

This function does NOT save the figures as images and reference them.

In [ ]:
md = result.document.export_to_markdown(page_break_placeholder="<!-- page break -->")
print(md)

In [ ]:
md_remove_img = md.replace("<!-- image -->\n\n", "")
print(md_remove_img)

In [ ]:
md_split = [pg.strip() for pg in md_remove_img.split("<!-- page break -->\n\n")]
for i, pg in enumerate(md_split):
    print(f"PAGE {i + 1}:\n")
    print(pg)